# Table of Contents
* [1. Data Cleaning](#1.-Data-Cleaning)
* [2. elsect_summary](#2.-elsect_summary)
	* [2.1 NaNs](#2.1-NaNs)
	* [2.2 Missleading column TOTAL_EXPENDITURE](#2.2-Missleading-column-TOTAL_EXPENDITURE)
	* [2.3 New feature: revenue per enrollment](#2.3-New-feature:-revenue-per-enrollment)
* [3. US Demographics](#3.-US-Demographics)
	* [3.1 Group county level data to states and years](#3.1-Group-county-level-data-to-states-and-years)
	* [3.2 Rename columns](#3.2-Rename-columns)
	* [3.3 Extract state level data only](#3.3-Extract-state-level-data-only)
	* [3.4 Melt year columns to single column](#3.4-Melt-year-columns-to-single-column)
	* [3.5 Combine three data sets into a single dataframe](#3.5-Combine-three-data-sets-into-a-single-dataframe)


# 1. Data Cleaning

In [1]:
import pandas as pd 

# 2. elsect_summary

In [ ]:
financeSum = './data/data_raw/elsect_summary.csv'
df = pd.read_csv(financeSum)

## 2.1 NaNs

In [ ]:
# drop 1992 data, all NaNs
df = df.dropna(how = 'any').reset_index(drop = True)

## 2.2 Missleading column TOTAL_EXPENDITURE

In [ ]:
# EXPENDITURE subcategories do not sum up to total, creat new columns
## Summed up expenditures from subcategories 
df['TOTAL_KNOWN_EXPENDITURE'] = df[[ 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE',
       'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].sum(axis = 1)
## The difference btw total expenditures and summed up expenditures
df['UNKNOWN_EXPENDITURE'] = df.TOTAL_EXPENDITURE - df.TOTAL_KNOWN_EXPENDITURE

## 2.3 New feature: revenue per enrollment

In [5]:
# New feature: REV_PER_ENROLL revenue per enrollment
df['REV_PER_ENROLL'] = df['TOTAL_REVENUE'] / df['ENROLL']

In [6]:
df.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,TOTAL_KNOWN_EXPENDITURE,UNKNOWN_EXPENDITURE,REV_PER_ENROLL
0,Alabama,1993,727716.0,2827391,331409,1729295,766687,2833433,1564558,794146,237222.0,204207,2800133.0,33300.0,3.885295
1,Alaska,1993,121156.0,1191398,176150,775829,239419,1126398,494917,433788,36291.0,135791,1100787.0,25611.0,9.833586
2,Arizona,1993,676297.0,3427976,318465,1415407,1694104,3623946,1578889,1000914,164083.0,680139,3424025.0,199921.0,5.068743
3,Arkansas,1993,311432.0,1346909,128196,771079,447634,1376067,782791,386526,68617.0,97824,1335758.0,40309.0,4.324890
4,California,1993,5129788.0,28043338,2151157,17064146,8828035,28110986,15281147,8914559,1608514.0,1944760,27748980.0,362006.0,5.466764


In [13]:
df.to_csv('./data/data_cleaned/edu_finance_cleaned.csv', index = False)

# 3. US Demographics

In [19]:
file1 = './data/data_raw/POP1990.csv'
file2 = './data/data_raw/POP2000.csv'
file3 = './data/data_raw/est_POP2010_2015.csv'
df_1990 = pd.read_csv(file1, usecols=['year', 'stname', 'tot_pop'])
df_2000 = pd.read_csv(file2, usecols=['year', 'stname', 'tot_pop'])
df_2010_est = pd.read_csv(file3, encoding = "ISO-8859-1", usecols=['STNAME','CTYNAME', 'POPESTIMATE2011', 
                                                                  'POPESTIMATE2012', 'POPESTIMATE2013',
                                                                  'POPESTIMATE2014', 'POPESTIMATE2015' ])
colnames = ['stname', 'ctyname', '2011', '2012', '2013', '2014', '2015']

## 3.1 Group county level data to states and years

In [20]:
df_1990 = df_1990.groupby(['stname', 'year']).sum().reset_index()
df_2000 = df_2000.groupby(['stname', 'year']).sum().reset_index()

## 3.2 Rename columns

In [21]:
df_2010_est.columns = colnames

## 3.3 Extract state level data only

In [22]:
df_2010_est = df_2010_est[df_2010_est.stname == df_2010_est.ctyname].drop('ctyname', axis = 1).reindex()

In [23]:
df_2010_est.head()

,stname,2011,2012,2013,2014,2015
0,Alabama,4801108,4816089,4830533,4846411,4858979
68,Alaska,722720,731228,737442,737046,738432
98,Arizona,6468732,6553262,6630799,6728783,6828065
114,Arkansas,2938538,2949499,2957957,2966835,2978204
190,California,37700034,38056055,38414128,38792291,39144818


## 3.4 Melt year columns to single column

In [24]:
df_2010_est = df_2010_est.melt(id_vars='stname', value_vars=['2011', '2012', '2013', '2014', '2015'],
                               var_name='year', value_name='tot_pop')
df_2010_est.head()

,stname,year,tot_pop
0,Alabama,2011,4801108
1,Alaska,2011,722720
2,Arizona,2011,6468732
3,Arkansas,2011,2938538
4,California,2011,37700034


## 3.5 Combine three data sets into a single dataframe

In [25]:
df_pop = pd.concat([df_1990, df_2000, df_2010_est], axis = 0).sort_values(['stname', 'year'])
df_pop.year = df_pop.year.astype(int)

In [27]:
df_pop.head(30)

,stname,year,tot_pop
0,Alabama,1990,4152384
1,Alabama,1991,4877290
2,Alabama,1992,4517525
3,Alabama,1993,4449453
4,Alabama,1994,4665300
5,Alabama,1995,4986229
6,Alabama,1996,5101961
7,Alabama,1997,4955683
8,Alabama,1998,4490646
9,Alabama,1999,4685143


In [28]:
df_pop.to_csv('./data/data_cleaned/us_demographics_cleaned.csv', index = False)

# USA General Data 

In [284]:
file = './data/data_raw/usa.csv'
df_usa = pd.read_csv(file)
df_usa.head()

,Region Name,Indicator,Indicator Name,Indicator - Note,Indicator - Source,Indicator - unit,Units,1990,1991,1992,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,United States,AAC,Persons Obtaining Legal Permanent Resident Status,Legal permanent residents (LPRs) are persons w...,U. S. Department of Homeland Security,Number of persons,Number of persons,NaN,NaN,NaN,...,1052415.0,1107126.0,1130818.0,1042625.0,1062040.0,1031631.0,990553.0,1016518.0,1051031.0,NaN
1,United States,AAD,Total Federal Government Expenditure,Include federal expenditures or obligation for...,"U. S. Census Bureau, Consolidated Federal Fund...",Millions of dollars,Millions of dollars,NaN,NaN,NaN,...,2556200.0,2792600.0,3238400.0,3276400.0,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,AAE,Number of Farms,"A farm is any establishment from which $1,000 ...",United States Department of Agruculture (USDA),Number,Number,NaN,NaN,NaN,...,2204950.0,2184500.0,2169660.0,2149520.0,2131240.0,2109810.0,2102010.0,2085000.0,2068000.0,2060000.0
3,United States,AAF,Total Employment,The BEA employment series for states and local...,U. S. Bureau of Economic Analysis (BEA),Thousands of Jobs,Thousands of Jobs,138330900.0,137612800.0,138166100.0,...,179885700.0,179639900.0,174233700.0,173034700.0,176278700.0,179081700.0,182408100.0,186168100.0,190195400.0,NaN
4,United States,AAH,Extended Mass Layoff Events,Total private nonfarm. Extended mass layoff is...,U. S. Bureau of Labor Statistics (BLS),Number of Events,Number of Events,NaN,NaN,NaN,...,5363.0,8259.0,11824.0,7247.0,6596.0,6500.0,NaN,NaN,NaN,NaN


In [285]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 34 columns):
Region Name           1712 non-null object
Indicator             1712 non-null object
Indicator Name        1712 non-null object
Indicator - Note      1453 non-null object
Indicator - Source    1660 non-null object
Indicator - unit      1712 non-null object
Units                 1712 non-null object
1990                  722 non-null float64
1991                  566 non-null float64
1992                  567 non-null float64
1993                  620 non-null float64
1994                  565 non-null float64
1995                  617 non-null float64
1996                  619 non-null float64
1997                  670 non-null float64
1998                  712 non-null float64
1999                  909 non-null float64
2000                  1330 non-null float64
2001                  1076 non-null float64
2002                  1224 non-null float64
2003                  1290 non

**There are a lot of NaNs**

## Clean up column names

In [286]:
df_usa.columns

Index(['Region Name', 'Indicator', 'Indicator Name', 'Indicator - Note',
       'Indicator - Source', 'Indicator - unit', 'Units', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')

In [287]:
df_usa = df_usa.drop(['Indicator - Source', 'Indicator - unit'], axis = 1)

In [293]:
df_usa.columns = df_usa.columns.str.replace(' -? ?', '_')
df_usa.columns = df_usa.columns.str.lower()

In [294]:
df_usa.head()

,region_name,indicator,indicator_name,indicator_note,units,1990,1991,1992,1993,1994,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,United States,AAC,Persons Obtaining Legal Permanent Resident Status,Legal permanent residents (LPRs) are persons w...,Number of persons,NaN,NaN,NaN,NaN,NaN,...,1052415.0,1107126.0,1130818.0,1042625.0,1062040.0,1031631.0,990553.0,1016518.0,1051031.0,NaN
1,United States,AAD,Total Federal Government Expenditure,Include federal expenditures or obligation for...,Millions of dollars,NaN,NaN,NaN,NaN,NaN,...,2556200.0,2792600.0,3238400.0,3276400.0,NaN,NaN,NaN,NaN,NaN,NaN
2,United States,AAE,Number of Farms,"A farm is any establishment from which $1,000 ...",Number,NaN,NaN,NaN,NaN,NaN,...,2204950.0,2184500.0,2169660.0,2149520.0,2131240.0,2109810.0,2102010.0,2085000.0,2068000.0,2060000.0
3,United States,AAF,Total Employment,The BEA employment series for states and local...,Thousands of Jobs,138330900.0,137612800.0,138166100.0,140774400.0,144196600.0,...,179885700.0,179639900.0,174233700.0,173034700.0,176278700.0,179081700.0,182408100.0,186168100.0,190195400.0,NaN
4,United States,AAH,Extended Mass Layoff Events,Total private nonfarm. Extended mass layoff is...,Number of Events,NaN,NaN,NaN,NaN,NaN,...,5363.0,8259.0,11824.0,7247.0,6596.0,6500.0,NaN,NaN,NaN,NaN


## Melt year columns to single column

In [269]:
df_usa.columns

Index(['region_name', 'indicator', 'indicator_name', 'units', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')

In [295]:
df_usa = df_usa.melt(id_vars=['region_name', 'indicator', 'indicator_name','indicator_note', 'units'], value_vars = ['1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016'], var_name = 'year', value_name = 'value')

In [296]:
df_usa.year = df_usa.year.astype(int)

In [297]:
df_usa.head()

,region_name,indicator,indicator_name,indicator_note,units,year,value
0,United States,AAC,Persons Obtaining Legal Permanent Resident Status,Legal permanent residents (LPRs) are persons w...,Number of persons,1990,NaN
1,United States,AAD,Total Federal Government Expenditure,Include federal expenditures or obligation for...,Millions of dollars,1990,NaN
2,United States,AAE,Number of Farms,"A farm is any establishment from which $1,000 ...",Number,1990,NaN
3,United States,AAF,Total Employment,The BEA employment series for states and local...,Thousands of Jobs,1990,138330900.0
4,United States,AAH,Extended Mass Layoff Events,Total private nonfarm. Extended mass layoff is...,Number of Events,1990,NaN


## Extract indicator / indicator name / units dictionary

In [298]:
import pickle

In [299]:
nameBook = df_usa[['indicator', 'indicator_name', 'indicator_note', 'units']]
nameBook = nameBook.drop_duplicates()

In [300]:
nameBook.head()

,indicator,indicator_name,indicator_note,units
0,AAC,Persons Obtaining Legal Permanent Resident Status,Legal permanent residents (LPRs) are persons w...,Number of persons
1,AAD,Total Federal Government Expenditure,Include federal expenditures or obligation for...,Millions of dollars
2,AAE,Number of Farms,"A farm is any establishment from which $1,000 ...",Number
3,AAF,Total Employment,The BEA employment series for states and local...,Thousands of Jobs
4,AAH,Extended Mass Layoff Events,Total private nonfarm. Extended mass layoff is...,Number of Events


In [301]:
nameBook.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33 entries, 0 to 32
Data columns (total 4 columns):
indicator         33 non-null object
indicator_name    33 non-null object
indicator_note    28 non-null object
units             33 non-null object
dtypes: object(4)
memory usage: 1.3+ KB


In [302]:
nameBook = nameBook.melt(id_vars = 'indicator', value_vars = ['indicator_name', 'indicator_note','units'], value_name = 'content', var_name = 'kind')
nameBook.head()

,indicator,kind,content
0,AAC,indicator_name,Persons Obtaining Legal Permanent Resident Status
1,AAD,indicator_name,Total Federal Government Expenditure
2,AAE,indicator_name,Number of Farms
3,AAF,indicator_name,Total Employment
4,AAH,indicator_name,Extended Mass Layoff Events


In [303]:
nameBook = nameBook.pivot(index = 'kind', columns = 'indicator', values = 'content')
nameBook.head()

indicator,AAC,AAD,AAE,AAF,AAG,AAH,AAI,AAK,AAL,AAM,...,ABT,ABU,ABV,ABW,ABX,ABY,ABZ,ACA,ACB,ACD
kind,,,,,,,,,,,,,,,,,,,,,
indicator_name,Persons Obtaining Legal Permanent Resident Status,Total Federal Government Expenditure,Number of Farms,Total Employment,Drop in Welfare Rolls,Extended Mass Layoff Events,Separations,Personal Income,Per Capita Personal Income,Disposable Personal Income,...,18- to 24-Year-Olds Who Were High School Compl...,Population 25 Years Old and Over Who Have Bach...,Enrollment in Public Elementary and Secondary ...,Average Daily Attendance in Public Elementary ...,Public Elementary and Secondary Teachers,Pupil/Teacher Ratios in Public Elementary and ...,Public High School Graduates,Current Expenditure per Pupil in Average Daily...,Crops Production,Land Area
indicator_note,Legal permanent residents (LPRs) are persons w...,Include federal expenditures or obligation for...,"A farm is any establishment from which $1,000 ...",The BEA employment series for states and local...,NaN,Total private nonfarm. Extended mass layoff is...,Separations indicate the number of employees l...,The personal income of an area is defined as t...,Calculated as the total personal income of the...,Disposable personal income is total personal i...,...,High school completers include diploma recipie...,NaN,"Actual Data: 1990 - 2013, Projection Data:2014...",NaN,Teachers reported in full-time equivalents (FTE),Teachers reported in full-time equivalents (FT...,"Actual Data: 1990 - 2013, Projection Data:2014...",NaN,Excluding horticulture. State value of product...,Total land area including non-agricultural
units,Number of persons,Millions of dollars,Number,Thousands of Jobs,Number of families on welfare,Number of Events,Number,Millions of dollars,dollars,Millions of dollars,...,Percent,Percent,Number,Number,Number,Rate,Number,dollars,Thousands of Dollars,Acres


In [304]:
namedict = nameBook.to_dict()
namedict.keys()

dict_keys(['AAC', 'AAD', 'AAE', 'AAF', 'AAG', 'AAH', 'AAI', 'AAK', 'AAL', 'AAM', 'AAN', 'AAQ', 'ABA', 'ABB', 'ABK', 'ABL', 'ABM', 'ABN', 'ABO', 'ABP', 'ABQ', 'ABR', 'ABS', 'ABT', 'ABU', 'ABV', 'ABW', 'ABX', 'ABY', 'ABZ', 'ACA', 'ACB', 'ACD'])

In [305]:
namedict['AAC']

{'indicator_name': 'Persons Obtaining Legal Permanent Resident Status',
 'indicator_note': 'Legal permanent residents (LPRs) are persons who have been granted lawful permanent residence in the United States. They are also known as “green card” recipients',
 'units': 'Number of persons'}

In [306]:
with open('./data/data_cleaned/usa_indicators_dict.pickle', 'wb') as f:
    pickle.dump(namedict, f)

## Remove indicator descriptions, only use shortcut

In [308]:
df_usa = df_usa.drop(['indicator_name','indicator_note', 'units'], axis = 1)
df_usa.head()

,region_name,indicator,year,value
0,United States,AAC,1990,NaN
1,United States,AAD,1990,NaN
2,United States,AAE,1990,NaN
3,United States,AAF,1990,138330900.0
4,United States,AAH,1990,NaN


In [309]:
df_usa.to_csv('./data/data_cleaned/usa_with_nan_no_pivot.csv')

## pivot indicators to columns 

In [310]:
df_usa.head()

,region_name,indicator,year,value
0,United States,AAC,1990,NaN
1,United States,AAD,1990,NaN
2,United States,AAE,1990,NaN
3,United States,AAF,1990,138330900.0
4,United States,AAH,1990,NaN


In [311]:
df_usa_pivot = df_usa.pivot_table(columns = 'indicator', index = ['region_name', 'year'], values = 'value').reset_index()
df_usa_pivot.columns.name = None
df_usa_pivot.head()

,region_name,year,AAC,AAD,AAE,AAF,AAG,AAH,AAI,AAK,...,ABT,ABU,ABV,ABW,ABX,ABY,ABZ,ACA,ACB,ACD
0,Alabama,1990,NaN,NaN,NaN,2047865.0,NaN,NaN,NaN,64070364.0,...,NaN,NaN,721806.0,683833.0,NaN,NaN,40485.0,3327.175800,NaN,NaN
1,Alabama,1991,NaN,NaN,NaN,2060099.0,NaN,NaN,NaN,67662101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3626.562725,NaN,NaN
2,Alabama,1992,NaN,NaN,NaN,2097425.0,NaN,NaN,NaN,72847428.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3616.000000,NaN,NaN
3,Alabama,1993,NaN,NaN,NaN,2158752.0,51910.0,NaN,NaN,76199461.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3761.125448,NaN,NaN
4,Alabama,1994,NaN,NaN,NaN,2180001.0,NaN,NaN,NaN,80680540.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4036.531519,NaN,NaN


In [312]:
df_usa_pivot.to_csv('./data/data_cleaned/usa_with_nan_pivot.csv')

# Test

In [313]:
df_usa.head()

,region_name,indicator,year,value
0,United States,AAC,1990,NaN
1,United States,AAD,1990,NaN
2,United States,AAE,1990,NaN
3,United States,AAF,1990,138330900.0
4,United States,AAH,1990,NaN


In [314]:
nameBook

indicator,AAC,AAD,AAE,AAF,AAG,AAH,AAI,AAK,AAL,AAM,...,ABT,ABU,ABV,ABW,ABX,ABY,ABZ,ACA,ACB,ACD
kind,,,,,,,,,,,,,,,,,,,,,
indicator_name,Persons Obtaining Legal Permanent Resident Status,Total Federal Government Expenditure,Number of Farms,Total Employment,Drop in Welfare Rolls,Extended Mass Layoff Events,Separations,Personal Income,Per Capita Personal Income,Disposable Personal Income,...,18- to 24-Year-Olds Who Were High School Compl...,Population 25 Years Old and Over Who Have Bach...,Enrollment in Public Elementary and Secondary ...,Average Daily Attendance in Public Elementary ...,Public Elementary and Secondary Teachers,Pupil/Teacher Ratios in Public Elementary and ...,Public High School Graduates,Current Expenditure per Pupil in Average Daily...,Crops Production,Land Area
indicator_note,Legal permanent residents (LPRs) are persons w...,Include federal expenditures or obligation for...,"A farm is any establishment from which $1,000 ...",The BEA employment series for states and local...,NaN,Total private nonfarm. Extended mass layoff is...,Separations indicate the number of employees l...,The personal income of an area is defined as t...,Calculated as the total personal income of the...,Disposable personal income is total personal i...,...,High school completers include diploma recipie...,NaN,"Actual Data: 1990 - 2013, Projection Data:2014...",NaN,Teachers reported in full-time equivalents (FTE),Teachers reported in full-time equivalents (FT...,"Actual Data: 1990 - 2013, Projection Data:2014...",NaN,Excluding horticulture. State value of product...,Total land area including non-agricultural
units,Number of persons,Millions of dollars,Number,Thousands of Jobs,Number of families on welfare,Number of Events,Number,Millions of dollars,dollars,Millions of dollars,...,Percent,Percent,Number,Number,Number,Rate,Number,dollars,Thousands of Dollars,Acres
